In [ ]:
import numpy as np
np.arange(-20,30,2)[np.arange(14,14+3)]

In [ ]:
import numpy as np
import h5py

class HDF5Store(object):
    """
    Simple class to append value to a hdf5 file on disc (usefull for building keras datasets)
    
    Params:
        datapath: filepath of h5 file
        dataset: dataset name within the file
        shape: dataset shape (not counting main/batch axis)
        dtype: numpy dtype
    
    Usage:
        hdf5_store = HDF5Store('/tmp/hdf5_store.h5','X', shape=(20,20,3))
        x = np.random.random(hdf5_store.shape)
        hdf5_store.append(x)
        hdf5_store.append(x)
        
    From https://gist.github.com/wassname/a0a75f133831eed1113d052c67cf8633
    """
    def __init__(self, datapath, dataset, shape, dtype=np.float32, compression="gzip", chunk_len=1):
        self.datapath = datapath
        self.dataset = dataset
        self.shape = shape
        self.dtype = dtype
        print(self.dataset)
        self.id = [0]*len(self.dataset)
        
        with h5py.File(self.datapath, mode='w') as h5f:
            for i in range(len(self.dataset)):
                self.dset = h5f.create_dataset(
                    self.dataset[i],
                    shape=(0, ) + self.shape[i],
                    maxshape=(None, ) + self.shape[i],
                    dtype=self.dtype[i],
                    compression=compression,
                    chunks=(chunk_len, ) + self.shape[i],)
    
    def append(self, values):
        with h5py.File(self.datapath, mode='a') as h5f:
            for i in range(len(self.dataset)):
                h5f[self.dataset[i]].resize((self.id[i] + values[i].shape[0], ) + self.shape[i])
                h5f[self.dataset[i]][-values[i].shape[0]:] = values[i]
                self.id[i] += values[i].shape[0]
                h5f.flush()
            
    def load_dataset(self, hdf5_path):        
            # Show keys            
            N_db = 26
            N_r = 4096
            N_m = 24
            dbs = np.arange(0,32,2)     
            data_idxs_train = np.array([])
            data_idxs_test = np.array([])
            for i in np.arange(N_m):
                for j in np.arange(10,10+len(dbs)):
                    ids = i*N_r*N_db+j*N_r
                    idx = np.arange(ids, ids+N_r)
                    dataset = h5py.File(hdf5_path, "r")
                    data = [dataset['X'][idx], dataset['Y'][idx]]
#                     [print(item) for item in dataset.items()]                    
                    self.append(data)

# test
hdf5_path = '/media/bognev/CE50072F50071DB9/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
hdf5_srore = HDF5Store('/tmp/GOLD_XYZ_OSC.0001_1024_truncated.hdf5',['X', 'Y'], shape=[(1024,2),(24,)], dtype=[np.float32, np.int32])
hdf5_srore.load_dataset(hdf5_path)

   
!ls -l /tmp | grep 'hdf5'

In [ ]:
# !ls -l /tmp
import h5py
with h5py.File('/tmp/GOLD_XYZ_OSC.0001_1024_truncated.hdf5', mode='r') as h5f:
    [print(item) for item in h5f.items()]

In [ ]:
shape = (1024,2)
print((None,)+ shape)
print((0, ) + shape)
print((0+4096, ) + shape)
print((0+4096, ) + (24,))

In [ ]:
dbs = np.arange(-20,32,2)   
print(dbs[np.arange(10,10+13)])

In [ ]:
dbs = np.arange(0,28,2)     
print(dbs)
print(len(dbs))

In [ ]:
import numpy as np
import h5py
hdf5_path = '/media/bognev/CE50072F50071DB9/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
N_db = 26
N_r = 4096
N_m = 24
dbs = np.arange(0,32,2)  
data_idxs_train = np.array([])
data_idxs_test = np.array([])
for i in np.arange(N_m):
    for j in np.arange(10,10+len(dbs)):
        ids = i*N_r*N_db+j*N_r
        idx = np.arange(ids, ids+N_r)
        dataset = h5py.File(hdf5_path, "r")        
#         print(dataset['X'][idx])        
        print(dataset['Y'][idx[0]])
        print(dataset['Z'][idx[0]])